In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('myproj').getOrCreate()

25/05/07 13:21:37 WARN Utils: Your hostname, SRI resolves to a loopback address: 127.0.1.1; using 172.26.29.67 instead (on interface eth0)
25/05/07 13:21:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 13:21:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/07 13:21:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/07 13:21:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/05/07 13:21:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/05/07 13:21:40 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/05/07 13:21:40 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


25/05/07 13:21:55 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
data = spark.read.csv('Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/titanic.csv',inferSchema=True,header=True)

In [5]:
data.printSchema()
data.columns


root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [7]:
my_final_data = my_cols.na.drop()

In [8]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [9]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [10]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [11]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [15]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [13]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')


In [16]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [19]:
fit_model=pipeline.fit(my_final_data)
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [20]:
results = fit_model.transform(test_titanic_data)

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [24]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [25]:
AUC = my_eval.evaluate(results)

In [26]:
AUC

0.7682937181663837